In [1]:
import os
import json
import numpy as np
from omegaconf import OmegaConf

In [2]:
exp_dir = "../../../logs/de/asr/evaluate"
folders = {
    "Fine-tuned": "version_4",
    "No AC": "version_8",
    "1e-5": "version_32",
    "1e-2": "version_26",
    "1e-1": "version_41",
}

In [3]:
# define which accent is standard, which are seen and which are unseen
classes = {
    "standard": ["de"],
    "seen": ["ch", "at"],
    "unseen": ["gb", "it", "de_al", "fr", "de_ni", "us", "ca", "ru"],
}

In [4]:
# load WERs of each file, for each accent
wers = dict()
for exp, folder in folders.items():
    wers[exp] = {k[5:-4]: v for k, v in json.load(open(f"{exp_dir}/{folder}/avg_wers.json")).items()}

In [5]:
# transform dictionary into a numpy array, where one axis represents the accents and another the experiments
experiments = list(wers.keys())
accents = list(wers[experiments[0]].keys())
n_words = np.array([wers[experiments[0]][acc]["n_words"] for acc in accents])
avg_wers = np.array([[wers[exp][acc]["avg_wer"] for exp in experiments] for acc in accents])

In [6]:
# compute means for each class, for each experiment
class_indices = {k: [accents.index(acc) for acc in v] for k, v in classes.items()}
class_means = {k: np.mean(avg_wers[indices], axis=0) for k, indices in class_indices.items()}

In [7]:
# print the avg. WERs and the means (overall and per class) as a markdown table
headers = ["Accent / Dialect"] + experiments
print((" | ").join(headers))
print(f"|{'---:|'*len(headers)}")

# print avg. WERs
for i in range(len(accents)):
    row = [accents[i]] + [f"{avg_wers[i,j]*100:.2f}" for j in range(len(experiments))]
    print((" | ").join(row))

# print overall means
row = ["mean"] + [f"{np.mean(avg_wers[:,i])*100:.2f}" for i in range(len(experiments))]
print((" | ").join(row))

# print class means
for key, value in class_means.items():
    row = [f"{key} mean"] + [f"{value[i]*100:.2f}" for i in range(len(experiments))]
    print((" | ").join(row))

# print worst avg. WER of each experiment
row = ["worst"] + [f"{v*100:.2f}" for v in np.max(avg_wers, axis=0)]
print((" | ").join(row))

# print the avg. of the three worst avg. WERs of each experiment
row = ["3-worst mean."] + [f"{np.mean(np.sort(avg_wers, axis=0)[-3:,i])*100:.2f}" for i in range(len(experiments))]
print((" | ").join(row))


Accent / Dialect | Fine-tuned | No AC | 1e-5 | 1e-2 | 1e-1
|---:|---:|---:|---:|---:|---:|
at | 1.80 | 1.75 | 1.85 | 1.63 | 1.58
gb | 4.11 | 4.03 | 4.68 | 3.87 | 3.79
it | 1.40 | 1.28 | 1.60 | 1.40 | 1.40
de_al | 1.16 | 1.10 | 1.32 | 1.11 | 1.13
fr | 2.68 | 2.63 | 2.84 | 2.22 | 2.20
de_ni | 4.31 | 4.08 | 4.51 | 4.11 | 4.17
ch | 3.11 | 2.79 | 3.11 | 2.71 | 2.71
de | 1.92 | 1.92 | 1.98 | 1.72 | 1.72
us | 4.69 | 4.86 | 5.68 | 4.69 | 4.56
ca | 3.70 | 3.62 | 3.27 | 3.36 | 3.01
ru | 2.97 | 2.84 | 2.88 | 2.30 | 2.39
mean | 2.89 | 2.81 | 3.07 | 2.65 | 2.61
standard mean | 1.92 | 1.92 | 1.98 | 1.72 | 1.72
seen mean | 2.45 | 2.27 | 2.48 | 2.17 | 2.15
unseen mean | 3.13 | 3.05 | 3.35 | 2.88 | 2.83
worst | 4.69 | 4.86 | 5.68 | 4.69 | 4.56
3-worst mean. | 4.37 | 4.32 | 4.96 | 4.22 | 4.17


In [8]:
# print the experiment folders of each experiment (both train and eval folders)
print("Experiment folders:\n")
for exp, folder in folders.items():
    eval_folder = os.path.join(exp_dir, folder)
    eval_config = OmegaConf.load(os.path.join(eval_folder, "hparams.yaml"))
    train_folder = f'../{eval_config.asr.ckpt.replace("/checkpoints/last.ckpt", "")}'
    print(f"- {exp}: training `{train_folder[3:]}`, evaluation `{eval_folder[3:]}`")

Experiment folders:

- Fine-tuned: training `logs/asr/train/version_12`, evaluation `logs/de/asr/evaluate/version_4`
- No AC: training `logs/ensemble/train/binary/b3/DAT/version_2`, evaluation `logs/de/asr/evaluate/version_8`
- 1e-5: training `logs/de/ensemble/train/binary/b10/DAT/version_10`, evaluation `logs/de/asr/evaluate/version_32`
- 1e-2: training `logs/de/ensemble/train/binary/b10/DAT/version_0`, evaluation `logs/de/asr/evaluate/version_26`
- 1e-1: training `logs/de/ensemble/train/binary/b10/DAT/version_46`, evaluation `logs/de/asr/evaluate/version_41`
